In [ ]:
!pip install wikipedia2vec
!pip install gensim

In [ ]:
!pip install nltk

In [ ]:
from wikipedia2vec import Wikipedia2Vec # as wiki2vec
import os
import pandas as pd
import numpy as np
import pickle
import gensim

import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

## 다음은 colab과 drive를 연동하는 과정입니다.

성공하려면 여러분의 '내 드라이브'에 '18-2 트랜드'가 추가되어 있어야 한다.

In [ ]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

In [ ]:
# After executing the cell above, Drive
# files will be present in "/content/drive/My Drive".
!ls "/content/drive/My Drive"

In [ ]:
!ls -ltr  # find directory

## 다음은 예제 코드입니다. 정상적으로 작동하면 결과 뜰 것임.


### Pretrained data는 두 개로 나눠져 있다.

Word : 내용 전체 포함. 소문자로만 되어 있음.

Entity : 표제만 포함. 대문자 그대로임.

In [ ]:
# MODEL_FILE = '/content/drive/My Drive/18-2 트랜드/공유자료/enwiki/win10/enwiki_20180420_win10_100d.pkl'
MODEL_FILE = '/content/drive/My Drive/성균관대학교/Pyneer/3. 프로젝트 운영/18-2 트랜드/공유자료/enwiki/win10/enwiki_20180420_win10_100d.pkl' # 재필: 제 드라이브에서는 주소가 달라서 임시로 지정합니다. 

In [ ]:
# 참고
model = Word2Vec.load_word2vec_format(MODEL_FILE, binary=True,norm_only=True)

#the model is loaded. It can be used to perform all of the tasks mentioned above.

# getting word vectors of a word
dog = model['dog']

#performing king queen magic
print(model.most_similar(positive=['woman', 'king'], negative=['man']))

#picking odd one out
print(model.doesnt_match("breakfast cereal dinner lunch".split()))

#printing similarity index
print(model.similarity('woman', 'man'))

In [ ]:
wiki2vec = Wikipedia2Vec.load(MODEL_FILE)

In [ ]:
wiki2vec.get_word_vector('machine')

In [ ]:
wiki2vec.most_similar(wiki2vec.get_word('machine'))

In [ ]:
wiki2vec.most_similar(wiki2vec.get_word('learning'), 5)

In [ ]:
wiki2vec.get_entity_vector('Scarlett Johansson')

In [ ]:
wiki2vec.most_similar(wiki2vec.get_word('yoda'), 5)

In [ ]:
wiki2vec.most_similar(wiki2vec.get_entity('Scarlett Johansson'), 5)

우리 거 테스트 해보기

In [ ]:
wiki2vec.get_word_vector('cloud')

In [ ]:
wiki2vec.most_similar(wiki2vec.get_word('cloud'), 20)  # 컴퓨팅 관련된 말들이 많이 나옴

In [ ]:
wiki2vec.most_similar(wiki2vec.get_entity('Deep learning'), 20)

In [ ]:
wiki2vec.most_similar(wiki2vec.get_entity('deep learning'), 1)  # 대/소문자 구분이 되어있다 ㅡㅡ

In [ ]:
wiki2vec.get_entity('Deep learning')  # 엔티티는 표제인듯, 소문자화하지 않음

In [ ]:
wiki2vec.get_entity('Paging delay')

In [ ]:
wiki2vec.get_word_vector('cloud')  # 모든 워드 벡터는 lowercase=True로 trained, bigram 처리 안되었다

In [ ]:
wiki2vec.get_word_vector('Cloud')

In [ ]:
wiki2vec.get_word_vector('cloud_computing')

## Binary file 열기 도전! 은 실패!

In [ ]:
fname = MODEL_FILE  # 앞서 사용한 bin 파일 지금은 pickle(pkl)로 되어있다

# 파일 내용을 일 바이트씩 읽어서 화면에 16진수로 출력하기
try:
  f = open(fname, 'rb')  # 파일 열기
  
  i=0
  while i < 10:  # 10개만 찍어보자, 다 찍으려면 True 넣어라
    s = f.read(1)
    if s == '': break
    print ('%02X' % int(ord(s)))  # 1바이트씩 출력
    i+=1
  
  
  f.close()  # 파일 닫기

except IOError:
  print ('파일을 열 수 없습니다.')

In [ ]:
with open(fname, 'rb') as file:
  data = pickle.load(file)

In [ ]:
import gensim.models.keyedvectors as word2vec
model = word2vec.KeyedVectors.load_word2vec_format(fname, binary=True)

In [ ]:
f.close()

# 핑클

## 키워드와 wiki 매칭해보기

In [ ]:
fname = '/content/drive/My Drive/18-2 트랜드/공유자료/파일/all_articles_df.p'
fname = '/content/drive/My Drive/성균관대학교/Pyneer/3. 프로젝트 운영/18-2 트랜드/공유자료/파일/all_articles_df.p'

In [ ]:
all_articles_df = pd.read_pickle(fname)
all_articles_df.head()

DF 전처리?
IEEE_scraping_collab by 최재필 참고

In [ ]:
all_articles_df['pubDate'].str.extract(r'(\d+-\d+-\d+)')

In [ ]:
all_articles_df['pubDate'] = pd.to_datetime(all_articles_df['pubDate'], format='%Y-%m-%d', errors='coerce')
all_articles_df = all_articles_df[~ all_articles_df['pubDate'].isnull()]
all_articles_from1990 = all_articles_df[all_articles_df.pubDate >= pd.to_datetime("1990-01-01")]

In [ ]:
quarterly_keywords = all_articles_from1990[['pubDate', 'keywords']].groupby(pd.Grouper(key='pubDate', freq="Q")).agg(sum) # quarterly. http://pandas.pydata.org/pandas-docs/stable/timeseries.html#offset-aliases 
quarterly_keywords = quarterly_keywords[quarterly_keywords['keywords'] != False]
quarterly_keywords = quarterly_keywords[quarterly_keywords.keywords.str.len() != 0]


In [ ]:
def remove_none_in_list(li):
    li = [x for x in li if x != '']
    return li

In [ ]:
quarterly_keywords.keywords = quarterly_keywords.keywords.apply(remove_none_in_list)

In [ ]:
quarterly_keywords.head()

In [ ]:
#quarterly_keywords.sort_values(by='pubDate') ##?

In [ ]:
def keywords_frequency(li):
#     c=0
    freq = {}
    
    for i in li:
        if i in freq.keys():
            freq[i] += 1
        else:
            freq[i] = 1
    
    for k, v in freq.items():
      if v > 10:  ## min count를 넣어보았다
        pass
      else:
        freq[k] = -1
        continue
      
    freq2 = {}

    for k, v in freq.items():
      if v != -1:
        freq2[k] = v

#     print(c) # 이것은 min count도 안나온 키워드들
    
    return freq2

In [ ]:
all_freq = keywords_frequency(a)

In [ ]:
len(all_freq.keys())

In [ ]:
quarterly_keywords.head()

In [ ]:
quarterly_keywords['frequency'] = quarterly_keywords.keywords.apply(keywords_frequency)

In [ ]:
quarterly_keywords.head()

In [ ]:
quarterly_keywords.loc['1995-03-31'].frequency

In [ ]:
d = quarterly_keywords.loc['2017-09-30'].relativeFrequency

In [ ]:
a = quarterly_keywords.keywords.sum()

In [ ]:
len(a)

In [ ]:
sort_d = sorted(d.items(), key=lambda x: x[1])

In [ ]:
sum(d.values()) / len(d.values())

In [ ]:
min(d.values())

In [ ]:
max(d.values())

In [ ]:
sort_d[:10]

In [ ]:
print(a)